# How to share state between threads

By default, state in a graph is scoped to that thread.
LangGraph also allows you to specify a "scope" for a given key/value pair that exists between threads.

In this notebook we will go through an example of how to construct and use such a graph.

## Create graph

In [7]:
from langgraph.graph.graph import START, END
from langgraph.graph.message import MessageGraph, add_messages, MessagesState
from langgraph.graph.state import StateGraph
from langgraph.store.memory import MemoryStore
from langgraph.managed.shared_value import SharedValue
from typing import TypedDict, Annotated, Any
import uuid
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver


class AgentState(MessagesState):
    # We use an info key to track information
    # This is scoped to a user_id, so it will be information specific to each user
    info: Annotated[dict, SharedValue.on("user_id")]


# We will give this as a tool to the agent
# This will let the agent call this tool to save a fact
class Info(TypedDict):
    fact: str


# This is the prompt we give the agent
# We will pass known info into the prompt
# We will tell it to use the Info tool to save more
prompt = """You are helpful assistant.

Here is what you know about the user:

<info>
{info}
</info>

Help out the user. If the user tells you things that you may want to remember for the future, save them using the `Info` tool."""


# We give the model access to the Info tool
model = ChatOpenAI().bind_tools([Info])


# Our first node - this will call the model
def call_model(state):
    # We get all facts and assemble them into a string
    facts = [d['fact'] for d in state['info'].values()]
    info = "\n".join(facts)
    # Format system prompt
    system_msg = prompt.format(info=info)
    # Call model
    response = model.invoke([{"role": "system", "content": system_msg}] + state['messages'])
    return {"messages": [response]}


# Routing function to decide what to do next
# If no tool calls, then we end
# If tool calls, then we update memory
def route(state):
    if len(state['messages'][-1].tool_calls) == 0:
        return END
    else:
        return "update_memory"


# This function is responsible for updating the memory
def update_memory(state):
    tool_calls = []
    memories = {}
    # Each tool call is a new memory to save
    for tc in state['messages'][-1].tool_calls:
        # We append ToolMessages (to pass back to the LLM)
        # This is needed because OpenAI requires each tool call be followed by a ToolMessage
        tool_calls.append({"role": "tool", "content": "Saved!", "tool_call_id": tc['id']})
        # We create a new memory from this tool call
        memories[str(uuid.uuid4())] = {"fact": tc['args']['fact']}
    # Return the messages and memories to update the state with
    return {"messages": tool_calls, "info": memories}


# This is the in memory checkpointer we will use
# We need this because we want to enable threads (conversations)
memory = MemorySaver()

# This is the in memory Key Value store
# This is needed to save the memories
kv = MemoryStore()

# Construct this relatively simple graph
graph = StateGraph(AgentState)
graph.add_node(call_model)
graph.add_node(update_memory)
graph.add_edge("update_memory", END)
graph.add_edge(START, "call_model")
graph.add_conditional_edges("call_model", route)
graph = graph.compile(checkpointer=memory, store=kv)

## Run graph on one thread

We can now run the graph on one thread and give it some information

In [8]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}

for update in graph.stream({"messages": [{"role": "user", "content": "hi"}]}, config, stream_mode="updates"):
    print(update)

for update in graph.stream({"messages": [{"role": "user", "content": "i like pizza"}]}, config, stream_mode="updates"):
    print(update)

for update in graph.stream({"messages": [{"role": "user", "content": "peperroni"}]}, config, stream_mode="updates"):
    print(update)

{'call_model': {'messages': [AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 185, 'total_tokens': 195}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65eca425-e789-433f-ba92-859dc77835f6-0', usage_metadata={'input_tokens': 185, 'output_tokens': 10, 'total_tokens': 195})]}}
{'call_model': {'messages': [AIMessage(content='Pizza is a popular choice! What is your favorite type of pizza topping?', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 205, 'total_tokens': 221}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7204558e-6864-4d71-b0e1-6b0389afaa66-0', usage_metadata={'input_tokens': 205, 'output_tokens': 16, 'total_tokens': 221})]}}
{'call_model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aHqramL

## Run graph on a different thread

We can now run the graph on a different thread and see that it remembers facts about the user

In [9]:
config = {"configurable": {"thread_id": "2", "user_id": "1"}}

for update in graph.stream({"messages": [{"role": "user", "content": "what should i have for dinner?"}]}, config, stream_mode="updates"):
    print(update)

{'call_model': {'messages': [AIMessage(content="How about having a delicious pepperoni pizza for dinner? It's a classic choice!", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 198, 'total_tokens': 216}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-090270eb-5732-4c02-a137-0fff3b777811-0', usage_metadata={'input_tokens': 198, 'output_tokens': 18, 'total_tokens': 216})]}}
